# This is Work in Progress (incomplete & undocumented)

# Raster reclassify, vectorize and clip
Given a value raster and a polygon clip layer, the goal is to get polygons reflecting reclassified categories of raster values, which are nicely clipped by a feature of the clip layer.

In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.rio.overview import get_maximum_overview_level
from rasterio.plot import show
import matplotlib.pyplot as plt
import pprint
import numpy as np

# Get to know your raster

In [ ]:
dem_r = './inData/demBernCas/dem_bern.tif'
with rasterio.open(dem_r, 'r') as r:
    pprint.pprint(r.profile) # pprint (pretty print) does the same as print, but prettier
    print('-'*30)
    print('Overview levels:', r.overviews(1)) # There is only 1 band, show overviews for it.
    print('-'*30)
    # Plot raster with colorbar, see:
    # https://stackoverflow.com/questions/61327088/rio-plot-show-with-colorbar
    fig, ax = plt.subplots()
    rshow = show(r, ax=ax)
    fig.colorbar(rshow.get_images()[0], ax=ax)

# Read and reclassify

In [ ]:
# Read the raster into memory as nested numpy array.
# Note: If the raster would not fit in memory, we could use a read window in read().
with rasterio.open(dem_r, 'r') as r:
    nodata = r.nodata # Keep track of nodata value for reclassification later
    r_meta = r.meta.copy() # Copy meta data to use later on output raster
    r_data = r.read(1) # Read band 1

# Reclassify the raster
# Use vectorized operations to use full speed potential of numpy.
r_data[r_data==nodata] = 0
r_data[(0 < r_data) & (r_data <= 500)] = 1
r_data[(500 < r_data) & (r_data <= 1000)] = 2
r_data[(1000 < r_data) & (r_data <= 1500)] = 3
r_data[(1500 < r_data) & (r_data <= 2000)] = 4
r_data[r_data > 2000] = 5

# Cast to signed unisgned int8 (0 to 255), because we only deal with a small number of integer categories.
# The resulting file is 25% of the original's size. This might well be premature optimization though.
r_data = r_data.astype('uint8')

# Quick sanity check:
print(f'Min/Max values: {r_data.min()} / {r_data.max()}')

# Create polygons

In [ ]:
from rasterio import features
from shapely.geometry import shape
import geopandas as gpd

In [ ]:
# Rasterio's features.shapes function vectorizes a raster by creating polygons from raster cells with same values. 
# The raster has noData=0, which is why we can use the raster data also as mask to avoid creating polygons for 
# cells with value 0. The function returns 2-tuples of a geojson describing the polygon and the associated cell value.
feature_generator = features.shapes(
    r_data, 
    mask=r_data, 
    transform=r_meta['transform']
)
# The polygon geojson is parsed into a shapely geometry object by shapely's shape function. The 2-tuple 
# structure is preserved to be fed to (geo)pandas' from_records function to create a Geodataframe.
records_generator = ((shape(geom), category) for geom, category in feature_generator)
gdf = gpd.GeoDataFrame.from_records(
    records_generator,
    columns=['geom', 'category'],
).set_geometry(
    col='geom',
    drop=True,
    crs=r_meta['crs'].to_string()
)

In [ ]:
gdf.explore()

# Clip

In [ ]:
clip_layer = gpd.read_file('./inData/admUnitsBernCas/bern_community_borders.shp')
clip_layer.explore()

In [ ]:
clip_mask = clip_layer.loc[clip_layer['NAME']=='Grindelwald'].geometry.unary_union
clip_mask

In [ ]:
gdf = gpd.clip(
    gdf,
    mask=clip_mask,
    keep_geom_type=True
)

In [ ]:
gdf.explore()

In [ ]:
gdf.to_file('./outData/reclassifiedAndClipped.shp')